In [13]:
!python -V
!pip -V
!python -c "import sys, pkgutil; print('numpy', pkgutil.find_loader('numpy') is not None); print('torch', pkgutil.find_loader('torch') is not None)"
!pip install --upgrade --no-deps timm pylibjpeg pylibjpeg-libjpeg pylibjpeg-openjpeg
!pip install --upgrade --no-deps pylibjpeg==2.1.0 pylibjpeg-libjpeg==2.3.0 pylibjpeg-openjpeg==2.5.0 || true
!pip check || true

Python 3.11.13
pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
numpy True
torch True
bigframes 2.12.0 requires google-cloud-bigquery-storage, which is not installed.
pylibjpeg-libjpeg 2.3.0 has requirement numpy<3.0,>=2.0, but you have numpy 1.26.4.
pylibjpeg-openjpeg 2.5.0 has requirement numpy<3.0,>=2.0, but you have numpy 1.26.4.
gensim 4.3.3 has requirement scipy<1.14.0,>=1.7.0, but you have scipy 1.15.3.
datasets 4.1.1 has requirement pyarrow>=21.0.0, but you have pyarrow 19.0.1.
onnx 1.18.0 has requirement protobuf>=4.25.1, but you have protobuf 3.20.3.
google-cloud-bigtable 2.32.0 has requirement google-api-core[grpc]<3.0.0,>=2.17.0, but you have google-api-core 1.34.1.
preprocessing 0.1.13 has requirement nltk==3.2.4, but you have nltk 3.9.1.
cesium 0.12.4 has requirement numpy<3.0,>=2.0, but you have numpy 1.26.4.
google-colab 1.0.0 has requirement google-auth==2.38.0, but you have google-auth 2.40.3.
google-colab 1.0.0 has requirement notebook==6.5.7

In [ ]:
import os, sys
os.kill(os.getpid(), 9)

In [1]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
import pydicom
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.cuda.amp import autocast, GradScaler
import timm
from tqdm import tqdm

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

In [2]:
DATA_PATH = "/kaggle/input/rsna-2022-cervical-spine-fracture-detection"
train_df = pd.read_csv(os.path.join(DATA_PATH, "train.csv"))
test_df = pd.read_csv(os.path.join(DATA_PATH, "test.csv"))
sample_sub = pd.read_csv(os.path.join(DATA_PATH, "sample_submission.csv"))
TRAIN_IMG_DIR = os.path.join(DATA_PATH, "train_images")

In [3]:
class CervicalSliceDataset(Dataset):
    def __init__(self, df, root, transform=None):
        self.df = df
        self.root = root
        self.transform = transform
        self.study_ids = df["StudyInstanceUID"].unique().tolist()
    def __len__(self):
        return len(self.study_ids)
    def __getitem__(self, idx):
        study = self.study_ids[idx]
        folder = os.path.join(self.root, study)
        files = sorted([f for f in os.listdir(folder) if f.endswith(".dcm")])
        if len(files) == 0:
            raise RuntimeError(f"No DICOM in {folder}")
        chosen = files[len(files)//2]
        path = os.path.join(folder, chosen)
        ds = pydicom.dcmread(path)
        try:
            arr = ds.pixel_array
        except Exception:
            ds.decompress()
            arr = ds.pixel_array
        if arr.ndim == 3:
            arr = arr[0]
        img = Image.fromarray(arr).convert("L")
        if self.transform:
            img = self.transform(img)
        row = self.df[self.df["StudyInstanceUID"]==study].iloc[0]
        labels = torch.zeros(8, dtype=torch.float32)
        labels[0] = row["patient_overall"]
        for i in range(1,8):
            labels[i] = row[f"C{i}"]
        return img, labels


train_transforms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomRotation(10),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])
train_ds = CervicalSliceDataset(train_df, TRAIN_IMG_DIR, transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)


## Modelo ViT

In [4]:
# Para pruebas cpu y para el final si usare gpu cuda
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=8)
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=1e-2)

In [5]:
scaler = GradScaler() if torch.cuda.is_available() else None

def train_epoch(model, loader, optimizer, criterion, device, scaler=None):
    model.train()
    running_loss = 0.0
    for imgs, labels in tqdm(loader):
        imgs = imgs.to(device, non_blocking=True)
        labels = labels.to(device, non_blocking=True)
        optimizer.zero_grad()
        if scaler is not None:
            with autocast():
                out = model(imgs)
                loss = criterion(out, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            out = model(imgs)
            loss = criterion(out, labels)
            loss.backward()
            optimizer.step()
        running_loss += loss.item() * imgs.size(0)
    return running_loss / len(loader.dataset)

for epoch in range(1,4):
    loss = train_epoch(model, train_loader, optimizer, criterion, device, scaler)
    print(f"epoch {epoch} loss {loss:.4f}")

/tmp/ipykernel_227/1633072353.py:1: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler() if torch.cuda.is_available() else None
  0%|          | 0/127 [00:00<?, ?it/s]/tmp/ipykernel_227/1633072353.py:11: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
100%|██████████| 127/127 [01:02<00:00,  2.04it/s]


epoch 1 loss 0.3788


100%|██████████| 127/127 [00:21<00:00,  5.90it/s]


epoch 2 loss 0.3671


100%|██████████| 127/127 [00:22<00:00,  5.76it/s]

epoch 3 loss 0.3615
